# 3 A First Context Manager Example

In [1]:
class LoggingContextManager:
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        return 

In [2]:
with LoggingContextManager() as x:
    print(x)

In [3]:
class LoggingContextManager:
    def __enter__(self):
        return 'You are in a with-block!'
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        return 

In [4]:
with LoggingContextManager() as x:
    print(x)

You are in a with-block!


In [5]:
class LoggingContextManager:
    def __enter__(self):
        print("LoggingContextManager.__enter__()")
        return 'You are in a with-block!'

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("LoggingContextManager.__exit__({}, {}, {})".format(exc_type, exc_val, exc_tb))
        return

# 4 __enter__()

In [6]:
with LoggingContextManager() as x:
    print(x)

LoggingContextManager.__enter__()
You are in a with-block!
LoggingContextManager.__exit__(None, None, None)


In [7]:
# with LoggingContextManager() as x:
#     raise ValueError("Something has gone wrong!")
#     print(x)
    
# LoggingContextManager.__enter__()
# LoggingContextManager.__exit__(<class 'ValueError'>, Something has gone wrong!, <traceback object at 0x000001E8331DBA48>)
# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# <ipython-input-7-600a50871e4b> in <module>
#       1 with LoggingContextManager() as x:
# ----> 2     raise ValueError("Something has gone wrong!")
#       3     print(x)

# ValueError: Something has gone wrong!

In [8]:
with open('important_data.txt', 'r') as f:
    data = f.read()

In [9]:
f = open('a_file', 'w')
with f as g:
    print(f is g)

True


# 5 __exit__()

In [10]:
class LoggingContextManager:
    def __enter__(self):
        print("LoggingContextManager.__enter__()")
        return 'You are in a with-block!'

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            print("LoggingContextManager.__exit__: normal exit detected.")
        else:
            print("LoggingContextManager.__exit__(type={}, value={}, traceback={})".format(exc_type, exc_val, exc_tb))

In [11]:
with LoggingContextManager():
    pass

LoggingContextManager.__enter__()
LoggingContextManager.__exit__: normal exit detected.


In [13]:
# with LoggingContextManager():
#     raise ValueError("Something has gone wrong!")
    
# LoggingContextManager.__enter__()
# LoggingContextManager.__exit__(type=<class 'ValueError'>, value=Something has gone wrong!, traceback=<traceback object at 0x00000282D250B548>)
# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# <ipython-input-12-4be171549525> in <module>
#       1 with LoggingContextManager():
# ----> 2     raise ValueError("Something has gone wrong!")

# ValueError: Something has gone wrong!

# 6 __exit__() and Exception Propagation

In [14]:
try:
    with LoggingContextManager() as x:
        raise ValueError("The system is down!")
except ValueError:
    print("*** ValueError detected ***")

LoggingContextManager.__enter__()
LoggingContextManager.__exit__(type=<class 'ValueError'>, value=The system is down!, traceback=<traceback object at 0x00000282D24EEC88>)
*** ValueError detected ***


# 8 contextlib.contextmanager

In [15]:
import contextlib
import sys

In [16]:
@contextlib.contextmanager
def logging_context_manager():
    print("LoggingContextManager.__enter__()")
    try:
        yield "You are in a with-block!"
    except Exception:
        print("logging_context_manager: exceptional exit", sys.exc_info())

In [17]:
with logging_context_manager() as x:
    print(x)

LoggingContextManager.__enter__()
You are in a with-block!


In [18]:
with logging_context_manager() as x:
    raise ValueError("Something has gone wrong!")

LoggingContextManager.__enter__()
logging_context_manager: exceptional exit (<class 'ValueError'>, ValueError('Something has gone wrong!',), <traceback object at 0x00000282D25769C8>)


In [19]:
@contextlib.contextmanager
def logging_context_manager():
    print("LoggingContextManager.__enter__()")
    try:
        yield "You are in a with-block!"
    except Exception:
        print("logging_context_manager: exceptional exit", sys.exc_info())
        raise

In [21]:
# with logging_context_manager() as x:
#     raise ValueError("Something has gone wrong!")
    
# LoggingContextManager.__enter__()
# logging_context_manager: exceptional exit (<class 'ValueError'>, ValueError('Something has gone wrong!',), <traceback object at 0x00000282D2576688>)
# ---------------------------------------------------------------------------
# ValueError                                Traceback (most recent call last)
# <ipython-input-20-bb79c427ff8d> in <module>
#       1 with logging_context_manager() as x:
# ----> 2     raise ValueError("Something has gone wrong!")

# ValueError: Something has gone wrong!

# 9 Multiple Context Managers

In [23]:
@contextlib.contextmanager
def nest_test(name):
    print('Entering', name)
    yield
    print('Exiting', name)

In [24]:
with nest_test('outer'), nest_test('inner'):
    print('BODY')

Entering outer
Entering inner
BODY
Exiting inner
Exiting outer


In [25]:
with nest_test('outer'):
    with nest_test('inner'):
        print('BODY')

Entering outer
Entering inner
BODY
Exiting inner
Exiting outer


In [26]:
@contextlib.contextmanager
def nest_test(name):
    print('Entering', name)
    yield name
    print('Exiting', name)

In [28]:
with nest_test('outer') as n1, nest_test('inner nested in ' + n1):
    print('BODY')

Entering outer
Entering inner nested in outer
BODY
Exiting inner nested in outer
Exiting outer


In [29]:
@contextlib.contextmanager
def propagater(name, propagate):
    try:
        yield
        print(name, 'exited normally.')
    except Exception:
        print(name, 'received an exception!')
        if propagate:
            raise

In [30]:
with propagater('outer', True), propagater('inner', False):
    raise TypeError("Cannot convert lead into gold.")

inner received an exception!
outer exited normally.


In [31]:
with propagater('outer', False), propagater('inner', True):
    raise TypeError("Cannot convert lead into gold.")

inner received an exception!
outer received an exception!


# 10 Don't Pass a List

In [33]:
# # list is not context manager
# with [nest_test('a'), nest_test('b')]:
#     pass

# ---------------------------------------------------------------------------
# AttributeError                            Traceback (most recent call last)
# <ipython-input-32-7fe6cb001c71> in <module>
# ----> 1 with [nest_test('a'), nest_test('b')]:
#       2     pass

# AttributeError: __enter__

In [34]:
with nest_test('a'), nest_test('b'):
    pass

Entering a
Entering b
Exiting b
Exiting a


In [35]:
with nest_test('a'),\
     nest_test('b'),\
     nest_test('c'):
    pass

Entering a
Entering b
Entering c
Exiting c
Exiting b
Exiting a
